In [ ]:
!pip install langdetect

In [96]:
import bz2
import numpy as np
import pandas as pd
import re
import string
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords

In [2]:
# Examine format of our data
file = '/content/train.ft.txt.bz2'

a = bz2.BZ2File(file)

b = 0
for i in a:
    x = i.decode(encoding='utf-8').strip()
    b += 1
    print(x)  # We can see label separated by space from the review
    if b == 5:
        break

__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^
__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.
__label__2 Amazing!: This soundtrack is m

In [3]:
# Write data extraction function
def get_data(file: str) -> pd.DataFrame:
    # Crete list to collect data
    data = list()
    with bz2.open(file, 'rt', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(' ', 1)  # separate by the first space
            # Check the amount of parts
            if len(parts) == 2:
                label, text = parts

                # Add the data to list
                data.append((label, text))

    # Create data frame
    df = pd.DataFrame(
        data,
        columns=['label', 'text']
    )
    df.label = df.label.str.extract(r'(\d+)').astype(int)  # convert label
    return df

In [4]:
amazon_reviews_data = get_data(file)

In [5]:
len(amazon_reviews_data)

3600000

In [233]:
# Create a class for text processing bf tokenization, stemming and lemmatization
class TextProcessing():

    # Download 'stopwords' to handle stopwords
    nltk.download('stopwords')
    stopword = stopwords.words('english')  # add English language

    def __init__(self):
        # Set patterns for re module to handle special symbols
        self.html_pattern = re.compile('<.*?>')
        self.url_pattern = re.compile(r'https?://\S+|www\.\S+')
        self.hashtag_pattern = re.compile(r'\@\w+|\#')
        self.punctuations = string.punctuation

    # Set the lowercase to text
    def to_lowercase(self, text):
        return str(text).lower()

    # Remove html tags
    def remove_html_tags(self, text):
        return self.html_pattern.sub('', text)

    # Remove links
    def remove_url(self, text):
        return self.url_pattern.sub('', text)

    # Remove hashtags
    def remove_hashtags(self, text):
        return self.hashtag_pattern.sub('', text)

    # Remove commas, dots, colons, etc.
    def remove_punctuation(self, text):
        return re.sub(f"[{re.escape(self.punctuations)}]", " ", text)

    # Correct spelling of the words
    def spell_correction(self, text):
        return ''.join(TextBlob(text))

    # Remove emojies
    def remove_emoji(self, text):
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

    # Remove stopwords
    def remove_stopwords(self, text):
        new_text = list()

        for i in text.split():
            if i in self.stopword:
                new_text.append('')
            else:
                new_text.append(i)
        x = new_text[:]
        new_text.clear()
        return ' '.join(x)

    def remove_digits(self, text):
        return re.sub(r'\d+', '', text)

    def remove_extra_spaces(self, text):
        return re.sub(r'\s+', ' ', text).strip()

    # Create an aggregating function to use all the functions above
    def cleaning(self, text):
        text = self.to_lowercase(text)
        text = self.remove_html_tags(text)
        text = self.remove_hashtags(text)
        text = self.remove_emoji(text)
        text = self.remove_url(text)
        text = self.remove_stopwords(text)
        text = self.spell_correction(text)
        text = self.remove_extra_spaces(text)
        text = self.remove_digits(text)
        text = self.remove_punctuation(text)
        return text  # return the result text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [234]:
cleaning = TextProcessing()

In [235]:
test_df = amazon_reviews_data.head(15)
a = test_df['text'].apply(cleaning.cleaning)
a.iloc[6]

'buyer beware  self published book  want know why  read paragraphs   star reviews must written ms  haddon s family friends  or perhaps  herself  can t imagine anyone reading whole thing  i spent evening book friend hysterics reading bits pieces one another  definitely bad enough entered kind  worst book  contest  can t believe amazon even sells kind thing  maybe offer th grade term paper  to kill mockingbird   a book quite sure ms  haddon never heard of  anyway  unless mood send book someone joke   stay far  far away one '